In [1]:
from bs4 import BeautifulSoup
import requests
from collections import defaultdict
import pandas as pd
import time

In [2]:
url = "http://www.capterra.com/project-management-software/"
r = requests.get(url)

soup = BeautifulSoup(r.content)

In [3]:
products = soup.findAll('div', {'class': 'card  zindex-hack  listing'})
print len(products)

457


In [6]:
# TODO: for each product, extract the link to the profile page
# TODO: crawl each profile page, put the URLs in urls object, instead of the shortcut shown below

urls = ['http://www.capterra.com/project-management-software/spotlight/79104/Smartsheet/Smartsheet%20com',
'http://www.capterra.com/project-management-software/spotlight/110832/Mavenlink/Mavenlink']

In [4]:
# We will populate this on the first iteration
cols = []

In [7]:
pages = {}
for url in urls:
    r = requests.get(url)
    pages[url] = r.text
    # Sleep for a second to not hammer the server
    time.sleep(1)

In [29]:
all_features

[]

In [30]:
rows = []
for url in urls:
    page = pages[url]
    soup = BeautifulSoup(page)
    name = soup.find('h1', {'itemprop': 'name'}).text
    print name
    # This should only run the first time
    has_feature = []
    missing_feature = []
    checklist = soup.findAll('ul', {'class': 'check-list'})
    close_to_price = checklist[1].findAll('div', {'class': 'cell seven-twelfths'})
    rawprice= close_to_price[0].text.replace('See pricing details', '').strip()
    # TODO: do some further extraction on this data
    lis = soup.findAll('li', {'class': 'ss-check'})
    for li in lis:
        has_feature.append(li.text)
    lis = soup.findAll('li', {'class': 'ss-delete color-gray'})
    for li in lis:
        missing_feature.append(li.text)
    has_feature_dict = defaultdict(int)
    for feature in has_feature:
        has_feature_dict[feature] = 1
    row = []
    row.append(name)
    if len(cols) == 0:
        all_features = has_feature
        all_features.extend(missing_feature)
        cols = ['product_name']
        cols.extend(all_features)
    for c in range(1, len(cols)):
        has = has_feature_dict[cols[c]]
        row.append(has)
    rows.append(row)

Smartsheet
Mavenlink


In [31]:
df = pd.DataFrame(rows)
df.columns = cols
df

,product_name
0,Smartsheet
1,Mavenlink
